
# 🤖 บทสรุป: การใช้ระบบหลายเอเจนต์สำหรับการสนับสนุนลูกค้าอัตโนมัติ

## 📖 บทนำ
ในบทเรียนนี้ เราจะสร้างระบบหลายเอเจนต์ (Multi-Agent System) เพื่อช่วยในการสนับสนุนลูกค้าอัตโนมัติ โดยใช้ **crewAI** ซึ่งเป็นกรณีการใช้งานที่น่าสนใจและมีประโยชน์มาก

## ❓ เหตุใดการสนับสนุนลูกค้าจึงเหมาะกับเอเจนต์
- **ความจำเป็นในการตอบคำถามลูกค้าอย่างรวดเร็วและถูกต้อง**  
  เอเจนต์ AI สามารถเข้าถึงเอกสาร ความรู้ และศูนย์ช่วยเหลือต่างๆ เพื่อให้คำตอบที่มีคุณภาพแก่ลูกค้า

## 🚀 เริ่มต้นการสร้างระบบ

1. **นำเข้าโมดูลที่จำเป็น**
   ```python
   from crewai import Agent, Task, Crew
   ```

2. **กำหนดโมเดล LLM**
   - ใช้โมเดล GPT-3.5 Turbo:
   ```python
   import os
   os.environ["OPENAI_MODEL_NAME"] = "gpt-3.5-turbo"
   ```

## 👥 การสร้างเอเจนต์

1. **สร้างเอเจนต์สนับสนุน (Support Agent)**
   - **บทบาท:** ผู้แทนสนับสนุนลูกค้าระดับอาวุโส
   - **เป้าหมาย:** เป็นมิตรและช่วยเหลือลูกค้าอย่างเต็มที่
   - **เรื่องราวพื้นหลัง:** ทำงานที่ crewAI และช่วยเหลือลูกค้าเฉพาะราย
   ```python
   support_agent = Agent(
       role="ผู้แทนสนับสนุนลูกค้าระดับอาวุโส",
       goal="ให้ความช่วยเหลือที่เป็นมิตรและมีประสิทธิภาพ",
       backstory="คุณทำงานที่ crewAI และกำลังช่วยเหลือลูกค้ารายหนึ่ง",
       allow_delegate=False
   )
   ```

2. **สร้างเอเจนต์ตรวจสอบคุณภาพ (Support QA Agent)**
   - **บทบาท:** ผู้เชี่ยวชาญด้านการประกันคุณภาพในการสนับสนุน
   - **เป้าหมาย:** ตรวจสอบความถูกต้องและคุณภาพของคำตอบ
   - **เรื่องราวพื้นหลัง:** ทำงานร่วมกับทีมสนับสนุนเพื่อให้คำตอบที่ดีที่สุดแก่ลูกค้า
   ```python
   support_qa_agent = Agent(
       role="ผู้เชี่ยวชาญด้านการประกันคุณภาพในการสนับสนุน",
       goal="ตรวจสอบและปรับปรุงคำตอบเพื่อให้แน่ใจว่ามีคุณภาพสูงสุด",
       backstory="คุณช่วยตรวจสอบคำตอบของทีมสนับสนุนเพื่อให้ลูกค้าได้รับการช่วยเหลือที่ดีที่สุด",
       allow_delegate=True
   )
   ```
   > **หมายเหตุ:** เอเจนต์ QA สามารถมอบหมายงานกลับไปยังเอเจนต์สนับสนุนได้ หากพบปัญหา

## ⚙️ ความสำคัญของบทบาทและการมอบหมายงาน
- เอเจนต์ตัดสินใจว่าจะมอบหมายงานเมื่อใด โดยพิจารณาจากความซับซ้อนของคำถาม
- ความยืดหยุ่นนี้เป็นข้อดีของแอปพลิเคชัน AI ที่สามารถตอบสนองตามสถานการณ์ได้

## 🛠️ การใช้เครื่องมือ (Tools), การตั้งขอบเขต (Guardrails), และความจำ (Memory)

1. **นำเข้าเครื่องมือจาก crewAI**
   ```python
   from crewai.tools import SerpAPITool, ScrapeWebsiteTool, WebsiteSearchTool
   ```
   - **SerpAPITool:** ใช้ค้นหาข้อมูลจาก Google
   - **ScrapeWebsiteTool:** ดึงข้อมูลจากเว็บไซต์ที่ระบุ
   - **WebsiteSearchTool:** ค้นหาเนื้อหาภายในเว็บไซต์ (RAG)

2. **การกำหนดเครื่องมือให้เอเจนต์**
   - ระดับเอเจนต์: เอเจนต์สามารถใช้เครื่องมือในทุกงาน
   - ระดับงาน: เอเจนต์จะใช้เครื่องมือเฉพาะเมื่อทำงานนั้นๆ

3. **ตัวอย่างการสร้างเครื่องมือ**
   ```python
   docs_scrape_tool = ScrapeWebsiteTool(website_url="https://docs.crewai.com/")
   ```

## 📋 การสร้างงาน (Tasks)

1. **งานแก้ไขคำถาม (Inquiry Resolution Task)**
   - **รายละเอียด:** ให้เอเจนต์สนับสนุนตอบคำถามของลูกค้า
   - **เครื่องมือที่ใช้:** docs_scrape_tool
   ```python
   inquiry_resolution_task = Task(
       description="ลูกค้า {customer} ได้ส่งคำถามมา: {inquiry}",
       expected_output="คำตอบที่ครบถ้วนและเป็นมิตร",
       agent=support_agent,
       tools=[docs_scrape_tool]
   )
   ```

2. **งานตรวจสอบคุณภาพ (Quality Assurance Review Task)**
   - **รายละเอียด:** ให้เอเจนต์ QA ตรวจสอบคำตอบของเอเจนต์สนับสนุน
   - **เครื่องมือที่ใช้:** ไม่มี
   ```python
   qa_review_task = Task(
       description="ตรวจสอบคำตอบที่ให้กับลูกค้า",
       expected_output="คำตอบที่ได้รับการปรับปรุงและถูกต้อง",
       agent=support_qa_agent
   )
   ```

## 🤝 การสร้าง Crew และใช้ความจำ
- **สร้าง Crew:**
   ```python
   crew = Crew(
       agents=[support_agent, support_qa_agent],
       tasks=[inquiry_resolution_task, qa_review_task],
       use_memory=True
   )
   ```

## 🚀 การเริ่มการทำงานของ Crew
- กำหนดค่าที่จะใช้แทนที่ในการทำงาน:
   ```python
   inputs = {
       "customer": "DeepLearning.AI",
       "person": "Andrew",
       "inquiry": "ต้องการความช่วยเหลือในการตั้งค่า Crew และเริ่มการทำงาน พร้อมกับใช้ความจำ"
   }

   crew.kickoff(inputs)
   ```

## 💬 ผลลัพธ์และการทำงานของเอเจนต์

- **เอเจนต์สนับสนุน:**
  - ใช้เครื่องมือ docs_scrape_tool เพื่อเข้าถึงเอกสาร
  - ให้คำตอบที่เป็นมิตรและครบถ้วนแก่ลูกค้า

- **เอเจนต์ QA:**
  - ตรวจสอบคำตอบของเอเจนต์สนับสนุน
  - หากพบปัญหา จะมอบหมายงานกลับเพื่อแก้ไข
  - ปรับปรุงคำตอบให้ดียิ่งขึ้น

- **การทำงานร่วมกัน:**
  - เอเจนต์ทั้งสองสื่อสารและทำงานร่วมกันเพื่อให้ได้คำตอบที่ดีที่สุด
  - ใช้ความจำในการจดจำข้อมูลระหว่างการทำงาน

## 📝 บทสรุป
เราได้สร้างระบบหลายเอเจนต์สำหรับการสนับสนุนลูกค้าอัตโนมัติ เอเจนต์สามารถทำงานร่วมกัน ใช้เครื่องมือ และความจำ เพื่อให้คำตอบที่มีคุณภาพแก่ลูกค้า การกำหนดบทบาท การใช้เครื่องมือที่เหมาะสม และการตั้งขอบเขต ช่วยให้เอเจนต์ทำงานได้อย่างมีประสิทธิภาพ

## 💡 ข้อเสนอแนะ
- ลองปรับแต่งเอเจนต์และเครื่องมือตามความต้องการของคุณ
- พิจารณาเพิ่มเอเจนต์หรือเครื่องมือเพิ่มเติมเพื่อปรับปรุงระบบ
- ใช้แนวคิดนี้ในการสร้างระบบอัตโนมัติอื่นๆ ที่คุณสนใจ


# L3: Multi-agent Customer Support Automation

In this lesson, you will learn about the six key elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [18]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, API and LLM

In [19]:
from crewai import Agent, Task, Crew

In [20]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Role Playing, Focus and Cooperation

In [21]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

- By not setting `allow_delegation=False`, `allow_delegation` takes its default value of being `True`.
- This means the agent _can_ delegate its work to another agent which is better suited to do a particular task. 

In [22]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* **Role Playing**: Both agents have been given a role, goal and backstory.
* **Focus**: Both agents have been prompted to get into the character of the roles they are playing.
* **Cooperation**: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [23]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
```

- Instantiate a document scraper tool.
- The tool will scrape a page (only 1 URL) of the CrewAI documentation.

In [24]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- You are passing the Tool on the Task Level.

In [25]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [26]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [27]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

2024-09-25 11:06:28,338 - 140220058565504 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [28]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to provide a detailed and informative response to Andrew Ng from DeepLearningAI regarding how to add memory to a Crew in crewAI.

Action: Read website content
Action Input: {"url": "https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"} 

crewAI
 crewAI
 Initializing search
 crewAI
 crewAI
 crewAI
 Home
 Getting Started
 Getting Started
 Installing CrewAI
 Starting a new CrewAI project
 Core Concepts
 Core Concepts
 Agents
 Tasks
 Tools
 Pro

Thought: I now can give a great answer

Final Answer: 

Dear Andrew Ng,

Thank you for reaching out with your question on how to add memory to a Crew in crewAI. To assist you with this, I have carefully reviewed the documentation on the crewAI website, specifically the section on creating a Crew and kicking it off.

To add memory to a Crew in crewAI, follow these step-by-step instructions:

1. Log in to your crewAI account using your credentials.
2. Once logged in, navigate to the "Settings" or "Account Settings" section of the platform.
3. Look for an option related to "Memory Management" or "Add Memory" within the settings.
4. Click on the "Add Memory" option, if available.
5. You may be prompted to select the amount of memory you want to add to your Crew. Choose the desired memory capacity based on your requirements.
6. Confirm the memory addition by following the on-screen instructions and entering any necessary payment information if applicable.
7. Once the memory upgrade is compl

- Display the final result as Markdown.

In [29]:
from IPython.display import Markdown
Markdown(result)

Dear Andrew Ng,

Thank you for reaching out with your question on how to add memory to a Crew in crewAI. To assist you with this, I have carefully reviewed the documentation on the crewAI website, specifically the section on creating a Crew and kicking it off.

To add memory to a Crew in crewAI, follow these step-by-step instructions:

1. Log in to your crewAI account using your credentials.
2. Once logged in, navigate to the "Settings" or "Account Settings" section of the platform.
3. Look for an option related to "Memory Management" or "Add Memory" within the settings.
4. Click on the "Add Memory" option, if available.
5. You may be prompted to select the amount of memory you want to add to your Crew. Choose the desired memory capacity based on your requirements.
6. Confirm the memory addition by following the on-screen instructions and entering any necessary payment information if applicable.
7. Once the memory upgrade is complete, you should see the updated memory capacity reflected in your Crew's settings.

Memory in crewAI allows you to store and access data across different tasks within your Crew, enabling seamless information sharing and continuity throughout the workflow. By following these steps, you can successfully add memory to your Crew in crewAI and enhance the overall performance and collaboration within your project.

If you require further assistance or detailed guidance on implementing memory in your Crew, please feel free to reach out. Our team at crewAI is dedicated to providing you with the best support and ensuring your success with our platform.

Best regards,

[Your Name]
Senior Support Representative
crewAI